### Multiling 2019 nmt evaluation

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import warnings
import os
import sys

sys.path.append("../")

warnings.filterwarnings("ignore")

In [2]:
data_path = "/home/lyn/lw/multiling2019/multiling2019_wiki/WikiTrain19/tagged_data/data/"

In [3]:
import pandas as pd

In [4]:
train_df = pd.read_csv(data_path + "train.csv", header=-1)
# train_df.to_csv(data_path + "train.csv", index=False)

In [5]:
valid_df = pd.read_csv(data_path + "valid.csv", header=-1)
# valid_df.to_csv(data_path + "valid.csv", index=False)

In [6]:
test_df = pd.read_csv(data_path + "test.csv", header=-1)
# test_df.to_csv(data_path + "test.csv", index=False)

In [7]:
train_df

,0,1
0,B_MISC I_MISC O O O O O O O O O O O O O O O O ...,El nombre con el que ha sido bautizada el área...
1,O O O O B_MISC O O O O O O O O,בשנות ה-60 וה-70 נפוצו פרשנויות המציגות את ספר...
2,B_MISC I_MISC I_MISC O O O O O O O,Den austerrikske arvefølgjekrigen (1740–1748) ...
3,O O O O O O O O B_MISC O O O O O O O O O O O O...,Though home to major cricketing venues and fra...
4,O O O O O B_MISC O O O O O O O O O O O O O O,In der Spätantike war der Neuplatonismus die e...
5,O B_MISC I_MISC I_MISC I_MISC O O O O O O O O ...,"The Directive Principles of State Policy, embo..."
6,B_MISC I_MISC I_MISC I_MISC O O O O O O O O O ...,"Luigi Maria Grignion de Montfort, ovvero Louis..."
7,B_MISC O O O O O O O O O O O O,Borat was also nominated for Best Adapted Scre...
8,O B_MISC I_MISC O O O O O O O O O O O O O O O ...,The Exelon Pavilions are four buildings that g...
9,O O O O O O O O O O O O O O O O O O O O O B_MI...,In den Jahren 1979 bis 2003 haben russische Sp...


### 1. Create data loaders

In [8]:
import os

train_path = data_path + "train.csv"
valid_path = data_path + "valid.csv"
test_path = data_path + "test.csv"

model_dir = " /home/lyn/lw/multiling2019/multiling2019_wiki/ner-bertdatadrive/multi_cased_L-12_H-768_A-12/"
init_checkpoint_pt = os.path.join(
    "/home/lyn/lw/multiling2019/multiling2019_wiki/ner-bert/datadrive/multi_cased_L-12_H-768_A-12/",
    "pytorch_model.bin")
bert_config_file = os.path.join(
    "/home/lyn/lw/multiling2019/multiling2019_wiki/ner-bert/datadrive/multi_cased_L-12_H-768_A-12/",
    "bert_config.json")
vocab_file = os.path.join(
    "/home/lyn/lw/multiling2019/multiling2019_wiki/ner-bert/datadrive/multi_cased_L-12_H-768_A-12/",
    "vocab.txt")

In [9]:
import torch
torch.cuda.set_device(0)
torch.cuda.is_available(), torch.cuda.current_device()

(True, 0)

In [10]:
from modules import BertNerData as NerData

In [11]:
data = NerData.create(train_path, valid_path, vocab_file, batch_size = 8)

In [12]:
len(data.train_dl.dataset), len(data.valid_dl.dataset)

(20157, 2479)

In [13]:
print(data.id2label)

['<pad>', '[CLS]', 'B_MISC', 'I_MISC', 'B_O', 'X']


In [14]:
sup_labels = ['B_MISC', 'I_MISC']

In [15]:
max([len(f.labels_ids) for f in data.train_dl.dataset])

424

### 2. Create model

In [16]:
from modules.models.bert_models import BertBiLSTMAttnNMT

In [20]:
# train a new model
model = BertBiLSTMAttnNMT.create(len(data.label2idx),
                                 bert_config_file,
                                 init_checkpoint_pt,
                                 enc_hidden_dim=256,
                                 dec_hidden_dim=256,
                                 dec_embedding_dim=128)

In [21]:
# load a model
# model.load_state_dict(torch.load("/home/lyn/lw/multiling2019/multiling2019_wiki/ner-bert/datadrive/models/multiling-2019/BertBiLSTMAttnNMT.cpt"))

In [22]:
model.get_n_trainable_params()

1908755

#### TODO: fix bug with len

### 3. Create Learner

In [23]:
from modules import NerLearner

In [24]:
num_epochs = 10
learner = NerLearner(
    model,
    data,
    best_model_path=
    "/home/lyn/lw/multiling2019/multiling2019_wiki/ner-bert/datadrive/models/multiling-2019/BertBiLSTMAttnNMT.cpt",
    lr=0.01,
    clip=1.0,
    sup_labels=sup_labels,
    t_total=num_epochs * len(data.train_dl))

### 4. Start learning

In [25]:
learner.fit(num_epochs, target_metric='prec')

INFO:root:Resuming train... Current epoch 0.
INFO:root:                                                                          
epoch 1, average train epoch loss=0.27203

INFO:root:on epoch 0 by max_prec: 0.733          
INFO:root:Saving new best model...


              precision    recall  f1-score   support

      B_MISC      0.666     0.741     0.702      2613
      I_MISC      0.762     0.720     0.740      1726

   micro avg      0.701     0.733     0.716      4339
   macro avg      0.714     0.730     0.721      4339
weighted avg      0.704     0.733     0.717      4339



INFO:root:                                                                          
epoch 2, average train epoch loss=0.07945

INFO:root:on epoch 0 by max_prec: 0.733          
train loss: 0.04523025080561638:   0%|          | 2/2520 [00:00<03:09, 13.25it/s]

              precision    recall  f1-score   support

      B_MISC      0.808     0.708     0.755      2613
      I_MISC      0.829     0.738     0.781      1726

   micro avg      0.817     0.720     0.765      4339
   macro avg      0.819     0.723     0.768      4339
weighted avg      0.816     0.720     0.765      4339



INFO:root:                                                                          
epoch 3, average train epoch loss=0.063785

INFO:root:on epoch 0 by max_prec: 0.733          
train loss: 0.02383701130747795:   0%|          | 1/2520 [00:00<04:27,  9.42it/s]

...

              precision    recall  f1-score   support

      B_MISC      0.813     0.805     0.809      2613
      I_MISC      0.826     0.814     0.820      1726

   micro avg      0.818     0.809     0.813      4339
   macro avg      0.820     0.810     0.815      4339
weighted avg      0.818     0.809     0.813      4339



INFO:root:                                                                           
epoch 9, average train epoch loss=0.012089

INFO:root:on epoch 5 by max_prec: 0.817          
train loss: 0.016600492876023054:   0%|          | 2/2520 [00:00<03:31, 11.92it/s]

              precision    recall  f1-score   support

      B_MISC      0.827     0.796     0.811      2613
      I_MISC      0.822     0.818     0.820      1726

   micro avg      0.825     0.805     0.815      4339
   macro avg      0.825     0.807     0.816      4339
weighted avg      0.825     0.805     0.815      4339



INFO:root:                                                                           
epoch 10, average train epoch loss=0.010983

INFO:root:on epoch 5 by max_prec: 0.817          


              precision    recall  f1-score   support

      B_MISC      0.821     0.802     0.811      2613
      I_MISC      0.829     0.811     0.820      1726

   micro avg      0.824     0.806     0.815      4339
   macro avg      0.825     0.807     0.816      4339
weighted avg      0.824     0.806     0.815      4339



### 5. Evaluate dev set

In [26]:
from modules.data.bert_data import get_bert_data_loader_for_predict
dl,f = get_bert_data_loader_for_predict(data_path + "valid.csv", learner)

In [27]:
learner.load_model()

In [28]:
preds = learner.predict(dl)

In [29]:
preds

[['[CLS]',
  'B_MISC',
  'X',
  'B_O',
  'B_O',
  'X',
  'B_O',
  'B_O',
  'X',
  'B_O',
  'B_O',
  'X',
  'X',
  'X',
  'X',
  'X',
  'X',
  'B_O',
  'B_O',
  'B_O',
  'B_O',
  'B_O',
  'B_O',
  'B_O',
  'X',
  'X',
  'B_O',
  'B_O',
  'B_O',
  'B_O',
  'B_O',
  'B_O',
  'B_O',
  'B_O',
  'X',
  'X'],
 ['[CLS]',
  'B_MISC',
  'I_MISC',
  'I_MISC',
  'B_O',
  'X',
  'B_O',
  'B_O',
  'B_O',
  'B_O',
  'B_O',
  'B_O',
  'X',
  'B_O',
  'B_O',
  'B_O',
  'X',
  'B_O',
  'B_O',
  'X',
  'B_O',
  'X',
  'B_O',
  'B_O',
  'B_O',
  'B_O',
  'B_O',
  'B_O',
  'B_O',
  'B_O',
  'B_O',
  'B_O'],
 ['[CLS]',
  'B_O',
  'X',
  'B_O',
  'B_MISC',
  'I_MISC',
  'B_O',
  'X',
  'X',
  'B_O',
  'B_O',
  'X'],
 ['[CLS]',
  'B_O',
  'B_MISC',
  'I_MISC',
  'I_MISC',
  'I_MISC',
  'B_O',
  'X',
  'B_O',
  'B_O',
  'X',
  'X',
  'B_O',
  'B_O',
  'B_O',
  'B_O',
  'B_O',
  'B_O',
  'B_O',
  'B_O',
  'B_O',
  'B_O',
  'B_O',
  'B_O',
  'B_O',
  'B_O',
  'B_O',
  'B_O',
  'X',
  'X'],
 ['[CLS]',
  'B_MISC',

# IOB precision

In [30]:
from modules.train.train import validate_step
print(
    validate_step(learner.data.valid_dl, learner.model, learner.data.id2label,
                  learner.sup_labels))

              precision    recall  f1-score   support

      B_MISC      0.810     0.802     0.806      2613
      I_MISC      0.782     0.840     0.810      1726

   micro avg      0.798     0.817     0.808      4339
   macro avg      0.796     0.821     0.808      4339
weighted avg      0.799     0.817     0.808      4339



# Span precision

In [31]:
from modules.utils.plot_metrics import get_bert_span_report
clf_report = get_bert_span_report(dl, preds, [])
print(clf_report)

              precision    recall  f1-score   support

           O      0.989     0.988     0.989     49548
        MISC      0.775     0.799     0.787      2613

    accuracy                          0.978     52161
   macro avg      0.882     0.893     0.888     52161
weighted avg      0.979     0.978     0.978     52161



### 6. Evaluate test set

In [32]:
from modules.data.bert_data import get_bert_data_loader_for_predict
dl,f = get_bert_data_loader_for_predict(data_path + "test.csv", learner)

In [33]:
preds = learner.predict(dl)

In [34]:
preds

[['[CLS]',
  'B_O',
  'B_MISC',
  'X',
  'X',
  'B_O',
  'X',
  'X',
  'B_O',
  'B_O',
  'B_O',
  'B_O',
  'X',
  'B_O',
  'B_O',
  'B_O',
  'B_O',
  'X',
  'X',
  'X',
  'X',
  'B_O',
  'B_O',
  'B_O',
  'B_O',
  'B_O',
  'B_O'],
 ['[CLS]',
  'B_MISC',
  'X',
  'X',
  'B_O',
  'B_O',
  'B_O',
  'X',
  'B_O',
  'B_O',
  'B_O',
  'X',
  'B_O',
  'X',
  'X',
  'B_O',
  'X',
  'B_O',
  'B_O',
  'X',
  'X'],
 ['[CLS]',
  'B_O',
  'X',
  'B_O',
  'B_O',
  'X',
  'X',
  'X',
  'X',
  'B_O',
  'B_O',
  'X',
  'X',
  'B_O',
  'X',
  'X',
  'X'],
 ['[CLS]',
  'B_MISC',
  'X',
  'X',
  'I_MISC',
  'X',
  'X',
  'B_O',
  'B_O',
  'B_O',
  'B_O',
  'B_O',
  'X',
  'X',
  'B_O',
  'X',
  'B_O',
  'B_O',
  'B_O',
  'B_O',
  'X',
  'B_O',
  'B_O',
  'X',
  'B_O',
  'B_O',
  'X',
  'B_O',
  'X',
  'B_O',
  'X',
  'X',
  'X',
  'B_O',
  'B_O',
  'B_O',
  'X',
  'B_O',
  'X',
  'X',
  'X',
  'B_O',
  'B_O'],
 ['[CLS]',
  'B_O',
  'B_O',
  'B_O',
  'B_O',
  'B_O',
  'B_MISC',
  'B_O',
  'B_O',
  'B_O',
 

In [35]:
len(preds)

2458

In [36]:
# for file,pred in zip(f,preds):
#     for i in range(len(pred)):
#         if pred[i] == 'B_MISC':
#             print(file.bert_tokens[i], end=' ')
#             for j in range(i+1,len(pred)):
#                 if pred[j] == 'I_MISC':
#                     print(file.bert_tokens[j], end= ' ')
#                 else:
#                     break
#             print("|",end = ' ')
#     print()

# IOB precision

In [37]:
from modules.train.train import validate_step
print(
    validate_step(learner.data.valid_dl, learner.model, learner.data.id2label,
                  learner.sup_labels))

              precision    recall  f1-score   support

      B_MISC      0.810     0.802     0.806      2613
      I_MISC      0.782     0.840     0.810      1726

   micro avg      0.798     0.817     0.808      4339
   macro avg      0.796     0.821     0.808      4339
weighted avg      0.799     0.817     0.808      4339



# Span precision

In [38]:
from modules.utils.plot_metrics import get_bert_span_report
clf_report = get_bert_span_report(dl, preds, [])
print(clf_report)

              precision    recall  f1-score   support

           O      0.988     0.987     0.988     49395
        MISC      0.755     0.780     0.767      2583

    accuracy                          0.976     51978
   macro avg      0.872     0.883     0.877     51978
weighted avg      0.977     0.976     0.977     51978

